In [1]:
%run load_dataset.ipynb
%run load_labels.ipynb

In [2]:
import scipy
import mne
import pandas as pd
import numpy as np

In [3]:
n_channels = 32
sfreq = 128
ch_types = 'eeg'

In [4]:
ch_names = [str(i) for i in range(n_channels)]

In [5]:
data = load_dataset(raw=True)
labels = load_labels()

# Filtering one sample

In [6]:
# sample = data[1]
# info = mne.create_info(ch_names = ch_names, sfreq = sfreq, ch_types=ch_types)
# raw = mne.io.RawArray(sample, info, first_samp=0, copy='auto', verbose=False)

In [7]:
# raw_highpass = raw.copy().filter(1, 50, verbose=False)

In [8]:
# ica = mne.preprocessing.ICA(n_components=18)
# ica.fit(raw_highpass)
# ica.plot_sources(raw_highpass)

In [9]:
#ica.plot_components()

In [16]:
# raw.plot(scalings='auto', n_channels=n_channels, verbose=False)

In [17]:
# raw_savgol = raw.copy().savgol_filter(h_freq=7, verbose=False)
# raw_savgol.plot(scalings='auto', n_channels=n_channels, verbose=False)

In [18]:
# raw_np = raw._data
# raw_savgol_np = raw_savgol._data
# trend_removed_np = raw_np-raw_savgol_np
# trend_removed = mne.io.RawArray(trend_removed_np, info, first_samp=0, copy='auto', verbose=False)
# trend_removed.plot(scalings='auto', n_channels=n_channels, verbose=False)

In [ ]:
# raw_bandpass = raw.copy().filter(2, 50, verbose=False)
# raw_bandpass.plot(scalings='auto', n_channels=32)

In [ ]:
# raw_bandpass_savgol = raw_bandpass.copy().savgol_filter(h_freq=7, verbose=False)
# raw_bandpass_savgol.plot(scalings='auto', n_channels=32)

In [9]:
filtered_data = np.empty(data.shape)

# Filtering the full dataset

In [7]:
for i, sample in enumerate(data):
    info = mne.create_info(ch_names = ch_names, sfreq = sfreq, ch_types=ch_types)
    raw = mne.io.RawArray(sample, info, first_samp=0, copy='auto', verbose=False)
    raw_bandpass = raw.copy().filter(2, 50, verbose=False)
    raw_bandpass_savgol = raw_bandpass.copy().savgol_filter(h_freq=7, verbose=False)
    filtered_data[i] = raw_bandpass_savgol._data